In [1]:
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

class_labels = trainset.classes

In [26]:
class DenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 128)
        self.fc2 = nn.Linear(128, 256)
        self.fc3 = nn.Linear(256, 512)
        self.fc4 = nn.Linear(512, 1024)
        self.fc5 = nn.Linear(1024, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.log_softmax(self.fc5(x), dim=1)
        return x

In [ ]:
# 모델 생성 및 옵티마이저 설정
net = DenseNet()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 모델 학습
for epoch in range(10):
    for images, labels in tqdm(trainloader):
        # 이미지 및 라벨 변환

        # 모델 실행 및 손실 계산
        outputs = net(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}: loss = {loss.item()}")



In [ ]:
# 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))


In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in class_labels}
total_pred = {classname: 0 for classname in class_labels}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[class_labels[label]] += 1
            total_pred[class_labels[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [40]:
# 간단한 CNN 모델 정의하기
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)  # 3개의 입력 채널, 32개의 출력 채널, 3x3 커널 크기, 패딩=1
        self.relu1 = nn.ReLU()  # ReLU 활성화 함수
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)  # 32개의 입력 채널, 64개의 출력 채널, 3x3 커널 크기, 패딩=1
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 커널 크기와 2의 스트라이드를 사용하는 Max Pooling
        self.flatten = nn.Flatten()  # 완전 연결 계층(FC Layer)을 통과하기 위한 텐서 평탄화
        self.fc1 = nn.Linear(64 * 8 * 8, 512)  # 64x8x8의 입력 특성, 512개의 출력 특성을 가지는 완전 연결층

        # 64: 두 번째 합성곱 레이어에서 출력된 채널 수입니다. 즉, 각 위치에서 64개의 특성 맵이 생성되었습니다.
        # 8x8: 두 번째 합성곱 레이어의 출력 특성맵의 공간 크기입니다. 이는 입력 이미지의 공간 해상도가 합성곱과 풀링 연산을 거치면서 감소한 결과입니다.

        self.relu3 = nn.ReLU()  # ReLU 활성화 함수
        self.fc2 = nn.Linear(512, 10)  # 512개의 입력 특성, 10개(레이블 개수) 출력 특성을 가지는 완전 연결층

    def forward(self, x):
        x = self.conv1(x)  # 입력: (N, 3, 32, 32), 출력: (N, 32, 32, 32) N은 배치 사이즈
        x = self.relu1(x)
        x = self.pool(x)  # 입력: (N, 32, 32, 32), 출력: (N, 32, 16, 16)
        x = self.conv2(x)  # 입력: (N, 32, 16, 16), 출력: (N, 64, 16, 16)
        x = self.relu2(x)
        x = self.pool(x)  # 입력: (N, 64, 16, 16), 출력: (N, 64, 8, 8)
        x = self.flatten(x)  # 2D 텐서로 평탄화, 입력: (N, 64, 8, 8), 출력: (N, 64*8*8)
        x = self.fc1(x)  # 입력: (N, 64*8*8), 출력: (N, 512)
        x = self.relu3(x)
        x = self.fc2(x)  # 입력: (N, 512), 출력: (N, 10)
        return x

In [41]:
# CNN 모델의 인스턴스 생성
model = CNN()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [ ]:

# 모델 학습
for epoch in range(10):
    for images, labels in tqdm(trainloader):
        # 이미지 및 라벨 변환

        # 모델 실행 및 손실 계산
        outputs = net(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # 옵티마이저 업데이트
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch + 1}: loss = {loss.item()}")

In [ ]:
# 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}")

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in class_labels}
total_pred = {classname: 0 for classname in class_labels}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[class_labels[label]] += 1
            total_pred[class_labels[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

# 디바이스 설정 (GPU 사용 가능하면 GPU, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 데이터셋에 대한 전처리 설정
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 무작위로 이미지 좌우 반전
    transforms.RandomCrop(32, padding=4),  # 무작위로 이미지 자르기 (32x32 크기로 자르고 패딩은 4로 설정)
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지를 정규화
])

# CIFAR-10 데이터셋 로드
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

# 데이터 로더 정의
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

# 사전 훈련된 ResNet 모델 불러오기
model = resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 10)  # CIFAR-10 분류를 위해 마지막의 fully connected layer 수정
model = model.to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 훈련 루프
num_epochs = 50
model.train()

for epoch in tqdm(range(num_epochs)):
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 100 == 99:
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], "
                  f"Loss: {running_loss / 100:.4f}, Accuracy: {100 * correct / total:.2f}%")
            running_loss = 0.0

# 테스트 루프
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

# 테스트셋의 정확도 출력
print(f"테스트 정확도: {100 * correct / total:.2f}%")